# Homework and bake-off: Relation extraction using distant supervision

In [55]:
__author__ = "Bill MacCartney and Christopher Potts"
__version__ = "CS224u, Stanford, Fall 2020"

## Contents

1. [Overview](#Overview)
1. [Set-up](#Set-up)
1. [Baselines](#Baselines)
  1. [Hand-build feature functions](#Hand-build-feature-functions)
  1. [Distributed representations](#Distributed-representations)
1. [Homework questions](#Homework-questions)
  1. [Different model factory [1 points]](#Different-model-factory-[1-points])
  1. [Directional unigram features [1.5 points]](#Directional-unigram-features-[1.5-points])
  1. [The part-of-speech tags of the "middle" words [1.5 points]](#The-part-of-speech-tags-of-the-"middle"-words-[1.5-points])
  1. [Bag of Synsets [2 points]](#Bag-of-Synsets-[2-points])
  1. [Your original system [3 points]](#Your-original-system-[3-points])
1. [Bake-off [1 point]](#Bake-off-[1-point])

## Overview

This homework and associated bake-off are devoted to developing really effective relation extraction systems using distant supervision. 

As with the previous assignments, this notebook first establishes a baseline system. The initial homework questions ask you to create additional baselines and suggest areas for innovation, and the final homework question asks you to develop an original system for you to enter into the bake-off.

## Set-up

See [the first notebook in this unit](rel_ext_01_task.ipynb#Set-up) for set-up instructions.

In [1]:
import os
import sys
module_paths = [os.path.abspath('../codes'), os.path.abspath('../..')]
for module_path in module_paths:
    if module_path not in sys.path:
        sys.path.append(module_path)

In [8]:
import numpy as np
import os
import rel_ext
from sklearn.linear_model import LogisticRegression
import utils

As usual, we unite our corpus and KB into a dataset, and create some splits for experimentation:

In [9]:
rel_ext_data_home = os.path.join('../../data', 'rel_ext_data')

In [10]:
corpus = rel_ext.Corpus(os.path.join(rel_ext_data_home, 'corpus.tsv.gz'))

In [11]:
kb = rel_ext.KB(os.path.join(rel_ext_data_home, 'kb.tsv.gz'))

In [12]:
dataset = rel_ext.Dataset(corpus, kb)

You are not wedded to this set-up for splits. The bake-off will be conducted on a previously unseen test-set, so all of the data in `dataset` is fair game:

In [13]:
splits = dataset.build_splits(
    split_names=['tiny', 'train', 'dev'],
    split_fracs=[0.01, 0.79, 0.20],
    seed=1)

In [14]:
splits

{'tiny': Corpus with 3,474 examples; KB with 445 triples,
 'train': Corpus with 263,285 examples; KB with 36,191 triples,
 'dev': Corpus with 64,937 examples; KB with 9,248 triples,
 'all': Corpus with 331,696 examples; KB with 45,884 triples}

## Baselines

### Hand-build feature functions

In [15]:
def simple_bag_of_words_featurizer(kbt, corpus, feature_counter):
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in ex.middle.split(' '):
            feature_counter[word] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for word in ex.middle.split(' '):
            feature_counter[word] += 1
    return feature_counter

In [16]:
kbt = kb.get_triples_for_relation('author')

In [17]:
corpus.examples[1]

Example(entity_1='New_Mexico', entity_2='Arizona', left='to all Spanish-occupied lands . The horno has a beehive shape and uses wood as the only heat source . The procedure still used in parts of', mention_1='New Mexico', middle='and', mention_2='Arizona', right='is to build a fire inside the Horno and , when the proper amount of time has passed , remove the embers and ashes and insert the', left_POS='to/TO all/DT Spanish-occupied/JJ lands/NNS ./. The/DT horno/NN has/VBZ a/DT beehive/NN shape/NN and/CC uses/VBZ wood/NN as/IN the/DT only/JJ heat/NN source/NN ./. The/DT procedure/NN still/RB used/VBN in/IN parts/NNS of/IN', mention_1_POS='New/NNP Mexico/NNP', middle_POS='and/CC', mention_2_POS='Arizona/NNP', right_POS='is/VBZ to/TO build/VB a/DT fire/NN inside/IN the/DT Horno/NNP and/CC ,/, when/WRB the/DT proper/JJ amount/NN of/IN time/NN has/VBZ passed/VBN ,/, remove/VB the/DT embers/NNS and/CC ashes/NNS and/CC insert/VB the/DT')

In [18]:
featurizers = [simple_bag_of_words_featurizer]

In [19]:
model_factory = lambda: LogisticRegression(fit_intercept=True, solver='liblinear')

In [20]:
baseline_results = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=featurizers,
    model_factory=model_factory,
    verbose=True)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.920      0.374      0.712        340       5716
author                    0.806      0.562      0.741        509       5885
capital                   0.700      0.221      0.488         95       5471
contains                  0.793      0.589      0.742       3904       9280
film_performance          0.790      0.576      0.735        766       6142
founders                  0.807      0.408      0.675        380       5756
genre                     0.651      0.165      0.409        170       5546
has_sibling               0.837      0.246      0.566        499       5875
has_spouse                0.852      0.330      0.647        594       5970
is_a                      0.679      0.229      0.488        497       5873
nationality               0.538      0.166      0.371        301       5677
parents     

Studying model weights might yield insights:

In [21]:
rel_ext.examine_model_weights(baseline_results)

Highest and lowest feature weights for relation adjoins:

     2.574 Córdoba
     2.447 Taluks
     2.420 Valais
     ..... .....
    -1.310 8
    -1.400 based
    -2.268 Earth

Highest and lowest feature weights for relation author:

     3.178 author
     2.543 books
     2.471 wrote
     ..... .....
    -2.558 controversial
    -2.867 infamous
    -2.998 1852

Highest and lowest feature weights for relation capital:

     3.643 capital
     1.840 city
     1.658 km
     ..... .....
    -1.080 present-day
    -1.095 Roman
    -1.130 and

Highest and lowest feature weights for relation contains:

     2.867 third-largest
     2.155 transferred
     2.137 population
     ..... .....
    -2.568 who
    -2.648 Sumatra
    -2.716 Midlands

Highest and lowest feature weights for relation film_performance:

     4.286 starring
     3.558 co-starring
     3.440 alongside
     ..... .....
    -2.032 Dhanush
    -2.043 Khakee
    -2.043 series

Highest and lowest feature weights for relation f

### Distributed representations

This simple baseline sums the GloVe vector representations for all of the words in the "middle" span and feeds those representations into the standard `LogisticRegression`-based `model_factory`. The crucial parameter that enables this is `vectorize=False`. This essentially says to `rel_ext.experiment` that your featurizer or your model will do the work of turning examples into vectors; in that case, `rel_ext.experiment` just organizes these representations by relation type.

In [22]:
GLOVE_HOME = os.path.join('data', 'glove.6B')

In [23]:
glove_lookup = utils.glove2dict(
    os.path.join(GLOVE_HOME, 'glove.6B.300d.txt'))

FileNotFoundError: [Errno 2] No such file or directory: 'data/glove.6B/glove.6B.300d.txt'

In [ ]:
def glove_middle_featurizer(kbt, corpus, np_func=np.sum):
    reps = []
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in ex.middle.split():
            rep = glove_lookup.get(word)
            if rep is not None:
                reps.append(rep)
    # A random representation of the right dimensionality if the
    # example happens not to overlap with GloVe's vocabulary:
    if len(reps) == 0:
        dim = len(next(iter(glove_lookup.values())))
        return utils.randvec(n=dim)
    else:
        return np_func(reps, axis=0)

In [ ]:
glove_results = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=[glove_middle_featurizer],
    vectorize=False, # Crucial for this featurizer!
    verbose=True)

With the same basic code design, one can also use the PyTorch models included in the course repo, or write new ones that are better aligned with the task. For those models, it's likely that the featurizer will just return a list of tokens (or perhaps a list of lists of tokens), and the model will map those into vectors using an embedding.

## Homework questions

Please embed your homework responses in this notebook, and do not delete any cells from the notebook. (You are free to add as many cells as you like as part of your responses.)

### Different model factory [1 points]

The code in `rel_ext` makes it very easy to experiment with other classifier models: one need only redefine the `model_factory` argument. This question asks you to assess a [Support Vector Classifier](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html).

__To submit:__ A wrapper function `run_svm_model_factory` that does the following: 

1. Uses `rel_ext.experiment` with the model factory set to one based in an `SVC` with `kernel='linear'` and all other arguments left with default values. 
1. Trains on the 'train' part of `splits`.
1. Assesses on the `dev` part of `splits`.
1. Uses `featurizers` as defined above. 
1. Returns the return value of `rel_ext.experiment` for this set-up.

The function `test_run_svm_model_factory` will check that your function conforms to these general specifications.

In [ ]:
from sklearn.svm import SVC
def run_svm_model_factory():
    ##### YOUR CODE HERE
    model_factory = lambda: SVC(kernel='linear')
    svc_model = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers= [simple_bag_of_words_featurizer],
    model_factory=model_factory,
    verbose=True)
    return svc_model

In [ ]:
def test_run_svm_model_factory(run_svm_model_factory):
    results = run_svm_model_factory()
    assert 'featurizers' in results, \
        "The return value of `run_svm_model_factory` seems not to be correct"
    # Check one of the models to make sure it's an SVC:
    assert 'SVC' in results['models']['adjoins'].__class__.__name__, \
        "It looks like the model factor wasn't set to use an SVC."

In [ ]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_run_svm_model_factory(run_svm_model_factory)

### Directional unigram features [1.5 points]

The current bag-of-words representation makes no distinction between "forward" and "reverse" examples. But, intuitively, there is big difference between _X and his son Y_ and _Y and his son X_. This question asks you to modify `simple_bag_of_words_featurizer` to capture these differences. 

__To submit:__

1. A feature function `directional_bag_of_words_featurizer` that is just like `simple_bag_of_words_featurizer` except that it distinguishes "forward" and "reverse". To do this, you just need to mark each word feature for whether it is derived from a subject–object example or from an object–subject example.  The included function `test_directional_bag_of_words_featurizer` should help verify that you've done this correctly.

2. A call to `rel_ext.experiment` with `directional_bag_of_words_featurizer` as the only featurizer. (Aside from this, use all the default values for `rel_ext.experiment` as exemplified above in this notebook.)

3. `rel_ext.experiment` returns some of the core objects used in the experiment. How many feature names does the `vectorizer` have for the experiment run in the previous step? Include the code needed for getting this value. (Note: we're partly asking you to figure out how to get this value by using the sklearn documentation, so please don't ask how to do it!)

In [ ]:
def directional_bag_of_words_featurizer(kbt, corpus, feature_counter):
    # Append these to the end of the keys you add/access in
    # `feature_counter` to distinguish the two orders. You'll
    # need to use exactly these strings in order to pass
    # `test_directional_bag_of_words_featurizer`.
    subject_object_suffix = "_SO"
    object_subject_suffix = "_OS"
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in ex.middle.split(' '):
            feature_counter[word + subject_object_suffix] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for word in ex.middle.split(' '):
            feature_counter[word + object_subject_suffix] += 1
    

    ##### YOUR CODE HERE


    return feature_counter


# Call to `rel_ext.experiment`:
##### YOUR CODE HERE
featurizers = [directional_bag_of_words_featurizer]
model_factory = lambda: LogisticRegression(fit_intercept=True, solver='liblinear')
rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=featurizers,
    model_factory=model_factory,
    verbose=True)

In [ ]:
def test_directional_bag_of_words_featurizer(corpus):
    from collections import defaultdict
    kbt = rel_ext.KBTriple(rel='worked_at', sbj='Randall_Munroe', obj='xkcd')
    feature_counter = defaultdict(int)
    # Make sure `feature_counter` is being updated, not reinitialized:
    feature_counter['is_OS'] += 5
    feature_counter = directional_bag_of_words_featurizer(kbt, corpus, feature_counter)
    expected = defaultdict(
        int, {'is_OS':6,'a_OS':1,'webcomic_OS':1,'created_OS':1,'by_OS':1})
    assert feature_counter == expected, \
        "Expected:\n{}\nGot:\n{}".format(expected, feature_counter)

In [ ]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_directional_bag_of_words_featurizer(corpus)

### The part-of-speech tags of the "middle" words [1.5 points]

Our corpus distribution contains part-of-speech (POS) tagged versions of the core text spans. Let's begin to explore whether there is information in these sequences, focusing on `middle_POS`.

__To submit:__

1. A feature function `middle_bigram_pos_tag_featurizer` that is just like `simple_bag_of_words_featurizer` except that it creates a feature for bigram POS sequences. For example, given 

  `The/DT dog/N napped/V`
  
   we obtain the list of bigram POS sequences
  
   `b = ['<s> DT', 'DT N', 'N V', 'V </s>']`. 
   
   Of course, `middle_bigram_pos_tag_featurizer` should return count dictionaries defined in terms of such bigram POS lists, on the model of `simple_bag_of_words_featurizer`.  Don't forget the start and end tags, to model those environments properly! The included function `test_middle_bigram_pos_tag_featurizer` should help verify that you've done this correctly.

2. A call to `rel_ext.experiment` with `middle_bigram_pos_tag_featurizer` as the only featurizer. (Aside from this, use all the default values for `rel_ext.experiment` as exemplified above in this notebook.)

In [ ]:
def middle_bigram_pos_tag_featurizer(kbt, corpus, feature_counter):

    ##### YOUR CODE HERE
    # split and regular expression to extract element after /
    # contruct the bigram
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in get_tag_bigrams(ex.middle_POS):
            feature_counter[word] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for word in get_tag_bigrams(ex.middle_POS):
            feature_counter[word] += 1

    return feature_counter


def get_tag_bigrams(s):
    """Suggested helper method for `middle_bigram_pos_tag_featurizer`.
    This should be defined so that it returns a list of str, where each
    element is a POS bigram."""
    # The values of `start_symbol` and `end_symbol` are defined
    # here so that you can use `test_middle_bigram_pos_tag_featurizer`.
    start_symbol = "<s>"
    end_symbol = "</s>"

    ##### YOUR CODE HERE
    tags = get_tags(s)
    if tags:
        first = "<s> " + tags[0]
        last =  tags[-1] + " </s>"
        result = [first]
        for i in range(len(tags) - 1):
            result.append(tags[i] + " " + tags[i + 1])
        
        return result + [last]
    else:
        return []




def get_tags(s):
    """Given a sequence of word/POS elements (lemmas), this function
    returns a list containing just the POS elements, in order.
    """
    return [parse_lem(lem)[1] for lem in s.strip().split(' ') if lem]


def parse_lem(lem):
    """Helper method for parsing word/POS elements. It just splits
    on the rightmost / and returns (word, POS) as a tuple of str."""
    return lem.strip().rsplit('/', 1)

# Call to `rel_ext.experiment`:
##### YOUR CODE HERE

featurizers = [middle_bigram_pos_tag_featurizer]
model_factory = lambda: LogisticRegression(fit_intercept=True, solver='liblinear')
rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=featurizers,
    model_factory=model_factory,
    verbose=True)

In [ ]:
def test_middle_bigram_pos_tag_featurizer(corpus):
    from collections import defaultdict
    kbt = rel_ext.KBTriple(rel='worked_at', sbj='Randall_Munroe', obj='xkcd')
    feature_counter = defaultdict(int)
    # Make sure `feature_counter` is being updated, not reinitialized:
    feature_counter['<s> VBZ'] += 5
    feature_counter = middle_bigram_pos_tag_featurizer(kbt, corpus, feature_counter)
    expected = defaultdict(
        int, {'<s> VBZ':6,'VBZ DT':1,'DT JJ':1,'JJ VBN':1,'VBN IN':1,'IN </s>':1})
    assert feature_counter == expected, \
        "Expected:\n{}\nGot:\n{}".format(expected, feature_counter)

In [ ]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_middle_bigram_pos_tag_featurizer(corpus)

### Bag of Synsets [2 points]

The following allows you to use NLTK's WordNet API to get the synsets compatible with _dog_ as used as a noun:

```
from nltk.corpus import wordnet as wn
dog = wn.synsets('dog', pos='n')
dog
[Synset('dog.n.01'),
 Synset('frump.n.01'),
 Synset('dog.n.03'),
 Synset('cad.n.01'),
 Synset('frank.n.02'),
 Synset('pawl.n.01'),
 Synset('andiron.n.01')]
```

This question asks you to create synset-based features from the word/tag pairs in `middle_POS`.

__To submit:__

1. A feature function `synset_featurizer` that is just like `simple_bag_of_words_featurizer` except that it returns a list of synsets derived from `middle_POS`. Stringify these objects with `str` so that they can be `dict` keys. Use `convert_tag` (included below) to convert tags to `pos` arguments usable by `wn.synsets`. The included function `test_synset_featurizer` should help verify that you've done this correctly.

2. A call to `rel_ext.experiment` with `synset_featurizer` as the only featurizer. (Aside from this, use all the default values for `rel_ext.experiment`.)

In [ ]:
from nltk.corpus import wordnet as wn

def synset_featurizer(kbt, corpus, feature_counter):

    ##### YOUR CODE HERE
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in ex.middle_POS.split(' '):
            syns = get_synsets(word)
            for syn in syns:
                feature_counter[syn] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for word in ex.middle_POS.split(' '):
            syns = get_synsets(word)
            for syn in syns:
                feature_counter[syn] += 1
    return feature_counter


def get_synsets(s):
    """Suggested helper method for `synset_featurizer`. This should
    be completed so that it returns a list of stringified Synsets
    associated with elements of `s`.
    """
    # Use `parse_lem` from the previous question to get a list of
    # (word, POS) pairs. Remember to convert the POS strings.
    wt = [parse_lem(lem) for lem in s.strip().split(' ') if lem]

    ##### YOUR CODE HERE
    synsets_elts = []
    if wt:
        synsets_elts = wn.synsets(wt[0][0], pos=convert_tag(wt[0][1]))
    return [str(elt) for elt in synsets_elts]




def convert_tag(t):
    """Converts tags so that they can be used by WordNet:

    | Tag begins with | WordNet tag |
    |-----------------|-------------|
    | `N`             | `n`         |
    | `V`             | `v`         |
    | `J`             | `a`         |
    | `R`             | `r`         |
    | Otherwise       | `None`      |
    """
    if t[0].lower() in {'n', 'v', 'r'}:
        return t[0].lower()
    elif t[0].lower() == 'j':
        return 'a'
    else:
        return None


# Call to `rel_ext.experiment`:
##### YOUR CODE HERE
rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=[synset_featurizer],
    verbose=True)




In [ ]:
def test_synset_featurizer(corpus):
    from collections import defaultdict
    kbt = rel_ext.KBTriple(rel='worked_at', sbj='Randall_Munroe', obj='xkcd')
    feature_counter = defaultdict(int)
    # Make sure `feature_counter` is being updated, not reinitialized:
    feature_counter["Synset('be.v.01')"] += 5
    feature_counter = synset_featurizer(kbt, corpus, feature_counter)
    # The full return values for this tend to be long, so we just
    # test a few examples to avoid cluttering up this notebook.
    test_cases = {
        "Synset('be.v.01')": 6,
        "Synset('embody.v.02')": 1
    }
    for ss, expected in test_cases.items():
        result = feature_counter[ss]
        assert result == expected, \
            "Incorrect count for {}: Expected {}; Got {}".format(ss, expected, result)

In [ ]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_synset_featurizer(corpus)

### Your original system [3 points]

There are many options, and this could easily grow into a project. Here are a few ideas:

- Try out different classifier models, from `sklearn` and elsewhere.
- Add a feature that indicates the length of the middle.
- Augment the bag-of-words representation to include bigrams or trigrams (not just unigrams).
- Introduce features based on the entity mentions themselves. <!-- \[SPOILER: it helps a lot, maybe 4% in F-score. And combines nicely with the directional features.\] -->
- Experiment with features based on the context outside (rather than between) the two entity mentions — that is, the words before the first mention, or after the second.
- Try adding features which capture syntactic information, such as the dependency-path features used by Mintz et al. 2009. The [NLTK](https://www.nltk.org/) toolkit contains a variety of [parsing algorithms](http://www.nltk.org/api/nltk.parse.html) that may help.
- The bag-of-words representation does not permit generalization across word categories such as names of people, places, or companies. Can we do better using word embeddings such as [GloVe](https://nlp.stanford.edu/projects/glove/)?

In the cell below, please provide a brief technical description of your original system, so that the teaching team can gain an understanding of what it does. This will help us to understand your code and analyze all the submissions to identify patterns and strategies. We also ask that you report the best score your system got during development, just to help us understand how systems performed overall.

In [ ]:
def middle_length_featurizer(kbt, corpus, feature_counter):
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        middle_length = len(ex.middle.split(' '))
        feature_counter[str(middle_length)] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        middle_length = len(ex.middle.split(' '))
        feature_counter[str(middle_length)] += 1
    return feature_counter

In [ ]:
def left_length_featurizer(kbt, corpus, feature_counter):
    left = '_left'
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        left_length = len(ex.left.split(' '))
        feature_counter[str(left_length) + left] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        left_length = len(ex.left.split(' '))
        feature_counter[str(left_length) + left] += 1
    return feature_counter

In [ ]:
def mention1_length_featurizer(kbt, corpus, feature_counter):
    mention1 = '_mention1'
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        mention1_length = len(ex.mention_1.split(' '))
        feature_counter[str(mention1_length) + mention1] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        mention1_length = len(ex.mention_1.split(' '))
        feature_counter[str(mention1_length) + mention1] += 1
    return feature_counter

In [ ]:
def mention2_length_featurizer(kbt, corpus, feature_counter):
    mention2 = '_mention2'
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        mention2_length = len(ex.mention_2.split(' '))
        feature_counter[str(mention2_length) + mention2] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        mention2_length = len(ex.mention_2.split(' '))
        feature_counter[str(mention2_length) + mention2] += 1
    return feature_counter

In [ ]:
def right_length_featurizer(kbt, corpus, feature_counter):
    right = '_right'
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        right_length = len(ex.right.split(' '))
        feature_counter[str(right_length) + right] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        right_length = len(ex.right.split(' '))
        feature_counter[str(right_length) + right] += 1
    return feature_counter

In [ ]:
def directional_middle_length_featurizer(kbt, corpus, feature_counter):
    subject_object_suffix = "_SO"
    object_subject_suffix = "_OS"
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        middle_length = len(ex.middle.split(' '))
        feature_counter[str(middle_length) + subject_object_suffix] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        middle_length = len(ex.middle.split(' '))
        feature_counter[str(middle_length) + object_subject_suffix] += 1
    return feature_counter

In [ ]:
def left_bag_of_words_featurizer(kbt, corpus, feature_counter):
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in ex.left.split(' '):
            feature_counter[word] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for word in ex.left.split(' '):
            feature_counter[word] += 1
    return feature_counter

In [ ]:
def right_bag_of_words_featurizer(kbt, corpus, feature_counter):
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in ex.right.split(' '):
            feature_counter[word] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for word in ex.right.split(' '):
            feature_counter[word] += 1
    return feature_counter

In [ ]:
def left_bigram_pos_tag_featurizer(kbt, corpus, feature_counter):
    left = '_left'
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in get_tag_bigrams(ex.left_POS):
            feature_counter[word + left] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for word in get_tag_bigrams(ex.left_POS):
            feature_counter[word + left] += 1
    return feature_counter

In [ ]:
def right_bigram_pos_tag_featurizer(kbt, corpus, feature_counter):
    right = '_right'
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in get_tag_bigrams(ex.right_POS):
            feature_counter[word + right] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for word in get_tag_bigrams(ex.right_POS):
            feature_counter[word + right] += 1

    return feature_counter

In [ ]:
def glove_mention1_featurizer(kbt, corpus, feature_counter):
    reps = []
    mention1= '_mention1'
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in ex.mention_1.split():
            rep = glove_lookup.get(word)
            if rep is not None:
                reps.append(rep)
    # A random representation of the right dimensionality if the
    # example happens not to overlap with GloVe's vocabulary:
    if len(reps) == 0:
        dim = len(next(iter(glove_lookup.values())))
        rand_vec = utils.randvec(n=dim)
        vec_dic =  dict(zip(range(len(rand_vec)), rand_vec))
        for feat, value in vec_dic.items():
            feature_counter[str(feat) + mention1] += value
    else:
        result = np.sum(reps, axis=0)
        result_dict = dict(zip(range(len(result)), result))
        for feat, value in result_dict.items():
            feature_counter[str(feat) + mention1] = value
    return feature_counter

In [ ]:
def glove_mention2_featurizer(kbt, corpus, feature_counter):
    reps = []
    mention2 = '_mention2'
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in ex.mention_2.split():
            rep = glove_lookup.get(word)
            if rep is not None:
                reps.append(rep)
    # A random representation of the right dimensionality if the
    # example happens not to overlap with GloVe's vocabulary:
    if len(reps) == 0:
        dim = len(next(iter(glove_lookup.values())))
        rand_vec = utils.randvec(n=dim)
        vec_dic =  dict(zip(range(len(rand_vec)), rand_vec))
        for feat, value in vec_dic.items():
            feature_counter[str(feat) + mention2] += value
    else:
        result = np.sum(reps, axis=0)
        result_dict = dict(zip(range(len(result)), result))
        for feat, value in result_dict.items():
            feature_counter[str(feat) + mention2] += value
    return feature_counter

In [ ]:
def glove_entity_featurizer(kbt, corpus, feature_counter):
    reps = []
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in ex.mention_2.split():
            rep = glove_lookup.get(word)
            if rep is not None:
                reps.append(rep)
        for word in ex.mention_1.split():
            rep = glove_lookup.get(word)
            if rep is not None:
                reps.append(rep)
    # A random representation of the right dimensionality if the
    # example happens not to overlap with GloVe's vocabulary:
    if len(reps) == 0:
        dim = len(next(iter(glove_lookup.values())))
        rand_vec = utils.randvec(n=dim)
        vec_dic =  dict(zip(range(len(rand_vec)), rand_vec))
        for feat, value in vec_dic.items():
            feature_counter[str(feat)] += value
    else:
        result = np.sum(reps, axis=0)
        result_dict = dict(zip(range(len(result)), result))
        for feat, value in result_dict.items():
            feature_counter[str(feat)] += value
    return feature_counter

In [ ]:
def glove_middle_featurizer(kbt, corpus, feature_counter):
    reps = []
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in ex.middle.split():
            rep = glove_lookup.get(word)
            if rep is not None:
                reps.append(rep)
    # A random representation of the right dimensionality if the
    # example happens not to overlap with GloVe's vocabulary:
    if len(reps) == 0:
        dim = len(next(iter(glove_lookup.values())))
        rand_vec = utils.randvec(n=dim)
        vec_dic =  dict(zip(range(len(rand_vec)), rand_vec))
        for feat, value in vec_dic.items():
            feature_counter[str(feat)] = value
    else:
        result = np.sum(reps, axis=0)
        result_dict = dict(zip(range(len(result)), result))
        for feat, value in result_dict.items():
            feature_counter[str(feat)] = value
    return feature_counter

In [ ]:
def directional_middle_bigram_pos_tag_featurizer(kbt, corpus, feature_counter):

    ##### YOUR CODE HERE
    # split and regular expression to extract element after /
    # contruct the bigram
    subject_object_suffix = "_SO"
    object_subject_suffix = "_OS"
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in get_tag_bigrams(ex.middle_POS):
            feature_counter[word + subject_object_suffix] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for word in get_tag_bigrams(ex.middle_POS):
            feature_counter[word + object_subject_suffix] += 1

    return feature_counter


In [ ]:
def directional_synset_featurizer(kbt, corpus, feature_counter):
    subject_object_suffix = "_SO"
    object_subject_suffix = "_OS"
    ##### YOUR CODE HERE
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in ex.middle_POS.split(' '):
            syns = get_synsets(word)
            for syn in syns:
                feature_counter[syn + subject_object_suffix] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for word in ex.middle_POS.split(' '):
            syns = get_synsets(word)
            for syn in syns:
                feature_counter[syn + object_subject_suffix] += 1
    return feature_counter

In [ ]:
featurizers = [left_bag_of_words_featurizer,
               left_length_featurizer,
               simple_bag_of_words_featurizer,
               middle_bigram_pos_tag_featurizer,
               directional_bag_of_words_featurizer,
               directional_middle_bigram_pos_tag_featurizer,
               directional_synset_featurizer,
               synset_featurizer, 
               middle_length_featurizer,
               directional_middle_length_featurizer,
               right_bag_of_words_featurizer,
               right_length_featurizer, 
               mention2_length_featurizer, 
               mention1_length_featurizer]


In [ ]:
# PLEASE MAKE SURE TO INCLUDE THE FOLLOWING BETWEEN THE START AND STOP COMMENTS:
#   1) Textual description of your system.
#   2) The code for your original system.
#   3) The score achieved by your system in place of MY_NUMBER.
#        With no other changes to that line.
#        You should report your score as a decimal value <=1.0
# PLEASE MAKE SURE NOT TO DELETE OR EDIT THE START AND STOP COMMENTS

# IMPORT ANY MODULES BELOW THE 'IS_GRADESCOPE_ENV' CHECK CONDITION. DOING
# SO ABOVE THE CHECK MAY CAUSE THE AUTOGRADER TO FAIL.

# START COMMENT: Enter your system description in this cell.
# The original system is built using logistic regression and a set of extracted features.
#The features are extracted from the corpus as follow:
#1 - left_bag_of_words_featurizer: finds all the corpus examples containing the two entities in the KBTriple, breaks the phrase appearing to the left of mention_1 into words, and counts the words.
#2 - left_length_featurizer: finds all the corpus examples containing the two entities in the KBTriple, breaks the phrase appearing to the left of mention_1 into words, and count the number of words, an additional label ‘_left’ is added to this feature.
#3 - simple_bag_of_words_featurizer:  finds all the corpus examples containing the two entities in the KBTriple, breaks the phrase appearing between the two entities into words, and counts the words.
#4 - middle_bigram_pos_tag_featurizer:  finds all the corpus examples containing the two entities in the KBTriple, extract the bigram POS of words appearing between the two entities, and counts each bigram POS.
#5 - directional_bag_of_words_featurizer: same as Simple_bag_of_words_featurizer but features are marked to indicate whether they are derived from a subject–object example or from an object–subject example.
#6 - directional_middle_bigram_pos_tag_featurizer: same as middle_bigram_pos_tag_featurizer but features are marked to indicate whether they are derived from a subject–object example or from an object–subject example.
#7 - synset_featurizer: finds all the corpus examples containing the two entities in the KBTriple, and synsets derived from middle_POS and counts them.
#8 - directional_synset_featurizer: same as synset_featurizer, but features are marked to indicate whether they are derived from a subject–object example or from an object–subject example.
#9 - middle_length_featurizer: finds all the corpus examples containing the two entities in the KBTriple, breaks the phrase appearing between the two entities into words, and count the number of words.
#10 - directional_middle_length_featurizer: same as middle_length_featurizer  but features are marked to indicate whether they are derived from a subject–object example or from an object–subject example.
#11 - right_bag_of_words_featurizer:  finds all the corpus examples containing the two entities in the KBTriple, breaks the phrase appearing to the right of mention_2 into words, and counts the words.
#12 - right_length_featurizer:  finds all the corpus examples containing the two entities in the KBTriple, breaks the phrase appearing to the right of mention_2 into words, and count the number of words, an additional label ‘_right’ is added to this feature.
#13 - mention2_length_featurizer:   finds all the corpus examples containing the two entities in the KBTriple, breaks the phrase appearing in mention_2 into words, and count the number of words, an additional label ‘_mention2’ is added to this feature.
#14 -  mention1_length_featurizer: finds all the corpus examples containing the two entities in the KBTriple, breaks the phrase appearing in mention_1 into words, and count the number of words, an additional label ‘_mention1’ is added to this feature.

#Once the features has been extracted
#1 - A logistic regression is trained using features from the ‘simple_bag_of_words_featurizer’: baseline performance is estimated
#2 - The final model is trained using logistic regression and all the features described above.

# My peak score was: 0.730
if 'IS_GRADESCOPE_ENV' not in os.environ:
    model_result = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=featurizers,
    verbose=True)
# STOP COMMENT: Please do not remove this comment.

In [24]:
rel_ext.find_new_relation_instances(
    dataset,
    featurizers=featurizers)

Highest probability examples for relation adjoins:

     1.000 KBTriple(rel='adjoins', sbj='Canada', obj='Vancouver')
     1.000 KBTriple(rel='adjoins', sbj='Vancouver', obj='Canada')
     1.000 KBTriple(rel='adjoins', sbj='Mexico', obj='Atlantic_Ocean')
     1.000 KBTriple(rel='adjoins', sbj='Atlantic_Ocean', obj='Mexico')
     1.000 KBTriple(rel='adjoins', sbj='Italy', obj='Sicily')
     1.000 KBTriple(rel='adjoins', sbj='Sicily', obj='Italy')
     1.000 KBTriple(rel='adjoins', sbj='Blue_Ridge_Mountains', obj='Appalachian_Mountains')
     1.000 KBTriple(rel='adjoins', sbj='Appalachian_Mountains', obj='Blue_Ridge_Mountains')
     1.000 KBTriple(rel='adjoins', sbj='Italy', obj='Palermo')
     1.000 KBTriple(rel='adjoins', sbj='Palermo', obj='Italy')

Highest probability examples for relation author:

     1.000 KBTriple(rel='author', sbj='Allen_Ginsberg', obj='Howl')
     1.000 KBTriple(rel='author', sbj='A_Christmas_Carol', obj='Charles_Dickens')
     1.000 KBTriple(rel='author', sbj=

     1.000 KBTriple(rel='parents', sbj='Gutzon_Borglum', obj='Lincoln_Borglum')
     1.000 KBTriple(rel='parents', sbj='Lincoln_Borglum', obj='Gutzon_Borglum')
     1.000 KBTriple(rel='parents', sbj='Philip_II_of_Macedon', obj='Alexander_the_Great')
     1.000 KBTriple(rel='parents', sbj='Alexander_the_Great', obj='Philip_II_of_Macedon')
     1.000 KBTriple(rel='parents', sbj='Thomas_Boleyn,_1st_Earl_of_Wiltshire', obj='Anne_Boleyn')
     1.000 KBTriple(rel='parents', sbj='Anne_Boleyn', obj='Thomas_Boleyn,_1st_Earl_of_Wiltshire')
     1.000 KBTriple(rel='parents', sbj='Prince_Philip,_Duke_of_Edinburgh', obj='Anne,_Princess_Royal')
     1.000 KBTriple(rel='parents', sbj='Anne,_Princess_Royal', obj='Prince_Philip,_Duke_of_Edinburgh')
     1.000 KBTriple(rel='parents', sbj='Charles,_Prince_of_Wales', obj='Prince_Philip,_Duke_of_Edinburgh')
     1.000 KBTriple(rel='parents', sbj='Prince_Philip,_Duke_of_Edinburgh', obj='Charles,_Prince_of_Wales')

Highest probability examples for relation p

## Bake-off [1 point]

For the bake-off, we will release a test set. The announcement will go out on the discussion forum. You will evaluate your custom model from the previous question on these new datasets using the function `rel_ext.bake_off_experiment`. Rules:

1. Only one evaluation is permitted.
1. No additional system tuning is permitted once the bake-off has started.

The cells below this one constitute your bake-off entry.

People who enter will receive the additional homework point, and people whose systems achieve the top score will receive an additional 0.5 points. We will test the top-performing systems ourselves, and only systems for which we can reproduce the reported results will win the extra 0.5 points.

Late entries will be accepted, but they cannot earn the extra 0.5 points. Similarly, you cannot win the bake-off unless your homework is submitted on time.

The announcement will include the details on where to submit your entry.

In [25]:
# Enter your bake-off assessment code in this cell.
# Please do not remove this comment.
if 'IS_GRADESCOPE_ENV' not in os.environ:
    
    # Please enter your code in the scope of the above conditional.
    ##### YOUR CODE HERE
    rel_ext_data_home_test = os.path.join(
    rel_ext_data_home, 'bakeoff-rel_ext-test-data')
    rel_ext.bake_off_experiment(model_result, rel_ext_data_home_test)


NameError: name 'model_result' is not defined

In [ ]:
# On an otherwise blank line in this cell, please enter
# your macro-average f-score (an F_0.5 score) as reported
# by the code above. Please enter only a number between
# 0 and 1 inclusive. Please do not remove this comment.
if 'IS_GRADESCOPE_ENV' not in os.environ:
    
    # Please enter your score in the scope of the above conditional.
    ##### YOUR CODE HERE
    0.726
